In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
# LSTM packages
from source import LstmParam as lstmP
from source import LstmNetwork as lstmN


In [ ]:
def generate_time_series(batch_size, n_steps, n_features):
    
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, n_features)
    
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10)) # wave 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + wave 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5) # + noise
    
    return series[..., np.newaxis].astype(np.float32)



In [ ]:
# X = np.array([[[]]])
# for i in range(10):
#     Y = np.array([[]])
#     for j in range(7):
#         arr = np.linspace(1,2,5)
#         print(arr)
#         Z = np.array([])
#         Z = np.append(Z, arr)
X = np.empty([3,5,7])
for sample in range(3):
    Y = np.empty([5,7])
    for time_step in range(5):
        arr = np.linspace(-(sample+time_step),sample**2+time_step**2+1,7)
        Y[time_step] = arr
    X[sample] = Y
print(X[0])

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
np.random.seed(0)


# Generating Time-series data
n_steps_X = 5 # time steps for data
n_steps_Y = 5 # time steps for target

n_samples = 50 # number of samples
n_features = 1 # number of features

series = generate_time_series(n_samples, n_steps_X + n_steps_Y, n_features)
series = scaler.fit_transform(series.reshape(-1, series.shape[-1])).reshape(series.shape)

X = scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
# Splitting the time series into train and test data
trainX, trainY = series[:, :n_steps_X], series[:, n_steps_Y:]
# trainX = X[:,:,:5]
# trainY = X[:,:, 5:]
print(trainX[0])

print(trainX.shape)
print(trainY.shape)
plt.plot(trainX[0,:,1], label = 'Train-X')
plt.plot(trainY[0,:,1], label = 'Train-Y')
plt.xlabel('Time')
plt.ylabel('Normalized value')
plt.legend()



In [ ]:
n_samples = len(trainX)
n_output = 1 # Dense layer output
n_neurons = 16
n_features = 5


# First setup the LSTM parameters (weights and biases)
lstm_param = lstmP.LstmParam(n_samples, n_neurons, n_features, n_output)

# Second, create LSTM network using LSTM parameters (weights and biases)
lstm_net = lstmN.LstmNetwork(lstm_param)



In [ ]:
loss_arr = []
arr = []

for epoch in range(200):
        
        
        # loop through the model for n_steps times
        # at each time step, it sends a data matrix (n_sample X n_features)
        
        for tim_ind in range(len(trainY[0])):  
                    
                 # forward pass for all time point once
                 # node_list is created here
                 pred_y = lstm_net.x_list_time_loop(trainX[:,tim_ind,:])
                
                 
                # print (pred_y)
                
        # back propagation after one complete forward pass        
        loss = lstm_net.y_list_is(trainY, lstmN.LossLayer) 
        
        
        loss_arr.append(loss)
        
       # print("loss:", "%.3e" % loss)
        
        # update the weights and biases
        lstm_param.apply_diff(lr=0.005)
        
        # clean up node_list after every epoch
        lstm_net.x_list_clear()
        
        

In [ ]:
#print (loss)
#print ((loss_arr))
plt.plot(loss_arr)

In [ ]:
# Testing using training data
# To see the training accuracy 


for tim_ind in range(len(trainY[0])):  
                    
                 # forward pass for all time point once
                 # node_list is created here
        pred_y = lstm_net.x_list_time_loop(trainX[:,tim_ind,:])
        
        
        
print (pred_y)

In [ ]:

# arr = scaler.inverse_transform(pred_y)
# trainY = scaler.inverse_transform(trainY)
    
plt.plot(pred_y[0],'b-*')
plt.plot (trainY[:,tim_ind],'r')